In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
import numpy as np
import seaborn as sns
from string import printable
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from statsmodels.formula.api import ols
from patsy import dmatrices
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_columns=100
pd.options.display.max_rows=100


In [35]:
share_df=pd.read_csv('data/share-of-population-urban.csv')

print(share_df.dtypes)
#Because the Urban_Population column is an object data type, there is probably a corrupt value
print(share_df.shape)
share_df.head()

Entity              object
Code                object
Year                 int64
Urban_Population    object
dtype: object
(15072, 4)


,Entity,Code,Year,Urban_Population
0,Afghanistan,AFG,1960,8.401
1,Afghanistan,AFG,1961,8.684
2,Afghanistan,AFG,1962,8.976
3,Afghanistan,AFG,1963,9.276
4,Afghanistan,AFG,1964,9.586


In [43]:
taxes_df=pd.read_csv('data/taxes-on-incomes-of-individuals-and-corporations-gdp.csv')
print(taxes_df.shape)
print(taxes_df.dtypes)
taxes_df.head(15)

(4871, 4)
Entity              object
Code                object
Year                 int64
Tax_Percent_GDP    float64
dtype: object


,Entity,Code,Year,Tax_Percent_GDP
0,Afghanistan,AFG,2003,0.165953
1,Afghanistan,AFG,2004,0.411647
2,Afghanistan,AFG,2005,0.320864
3,Afghanistan,AFG,2006,1.261181
4,Afghanistan,AFG,2007,1.323461
5,Afghanistan,AFG,2008,1.664719
6,Afghanistan,AFG,2009,2.608631
7,Afghanistan,AFG,2010,2.563413
8,Afghanistan,AFG,2011,2.573097
9,Afghanistan,AFG,2012,2.477800


## Data Cleaning

the Urban_Population column in the share_df is an object data type, meaning there is probably a corrupt value/float, since we need numeric values

In [70]:
share_df.Urban_Population.astype('float64')

ValueError: could not convert string to float: '88%'

In [80]:
type('88%')

str

In [83]:
share_df[share_df['Urban_Population'] =='88%']

,Entity,Code,Year,Urban_Population
220,American Samoa,ASM,2006,88%


In [86]:
#This shows that the % sign is implied in the remainder of the column
share_df[share_df['Entity'] =='American Samoa']

,Entity,Code,Year,Urban_Population
174,American Samoa,ASM,1960,66.211
175,American Samoa,ASM,1961,66.641
176,American Samoa,ASM,1962,67.068
177,American Samoa,ASM,1963,67.493
178,American Samoa,ASM,1964,67.916
179,American Samoa,ASM,1965,68.334
180,American Samoa,ASM,1966,68.75
181,American Samoa,ASM,1967,69.163
182,American Samoa,ASM,1968,69.574
183,American Samoa,ASM,1969,69.98


In [90]:
share_df.Urban_Population = share_df.Urban_Population\
.apply(lambda x: float(x[:-1]) if type(x)=='str' and '%' in x else float(x))
share_df.dtypes

Entity               object
Code                 object
Year                  int64
Urban_Population    float64
dtype: object

## Merging the two datasets

In [91]:
#Inner Joins join only shared keys. Here a combination of the Code column and Year column provides the key.
df = taxes_df.merge(share_df, on=['Code','Year'], how='inner')
print(df.shape)
df.head(20)

(4827, 6)


,Entity_x,Code,Year,Tax_Percent_GDP,Entity_y,Urban_Population
0,Afghanistan,AFG,2003,0.165953,Afghanistan,22.353
1,Afghanistan,AFG,2004,0.411647,Afghanistan,22.500
2,Afghanistan,AFG,2005,0.320864,x,22.703
3,Afghanistan,AFG,2006,1.261181,Afghanistan,22.907
4,Afghanistan,AFG,2007,1.323461,Afghanistan,23.113
5,Afghanistan,AFG,2008,1.664719,Afghanistan,23.320
6,Afghanistan,AFG,2009,2.608631,Afghanistan,23.528
7,Afghanistan,AFG,2010,2.563413,Afghanistan,23.737
8,Afghanistan,AFG,2011,2.573097,Afghanistan,23.948
9,Afghanistan,AFG,2012,2.477800,Afghanistan,24.160


In [92]:
#Column name Clean_up
df['Entity']=df['Entity_x']
df=df[['Entity', 'Code', 'Year', 'Tax_Percent_GDP',
       'Urban_Population']]
df.head()

,Entity,Code,Year,Tax_Percent_GDP,Urban_Population
0,Afghanistan,AFG,2003,0.165953,22.353
1,Afghanistan,AFG,2004,0.411647,22.500
2,Afghanistan,AFG,2005,0.320864,22.703
3,Afghanistan,AFG,2006,1.261181,22.907
4,Afghanistan,AFG,2007,1.323461,23.113


In [93]:
#Save merged to .csv
df.to_csv('data/merged_social_data.csv')

Linear Regression

In [94]:
from statsmodels.formula.api import ols

In [55]:
import statsmodels.api as sm

In [95]:
df.dtypes

Entity               object
Code                 object
Year                  int64
Tax_Percent_GDP     float64
Urban_Population    float64
dtype: object

In [96]:
lr_model = ols(formula='Tax_Percent_GDP~'+'Urban_Population', data=df).fit()
lr_model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Tax_Percent_GDP   R-squared:                       0.189
Model:                            OLS   Adj. R-squared:                  0.189
Method:                 Least Squares   F-statistic:                     1127.
Date:                Sun, 17 Jan 2021   Prob (F-statistic):          3.38e-222
Time:                        22:33:17   Log-Likelihood:                -14029.
No. Observations:                4827   AIC:                         2.806e+04
Df Residuals:                    4825   BIC:                         2.808e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept            1.6963      0.158     10.751      0.000       1.387       2.006
Urban_Population     0.0888      0.003     33.567      0.000       0.084       0.094
==============================================================================
Omnibus:                      888.216   Durbin-Watson:                   0.113
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2171.610
Skew:                           1.021   Prob(JB):                         0.00
Kurtosis:                       5.574   Cond. No.                         148.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Tax_Percent_GDP   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.549
Method:                 Least Squares   F-statistic:                     2.283
Date:                Sun, 17 Jan 2021   Prob (F-statistic):           1.47e-15
Time:                        22:05:11   Log-Likelihood:                -5430.3
No. Observations:                4827   AIC:                         2.002e+04
Df Residuals:                     246   BIC:                         4.972e+04
Df Model:                        4580                                         
Covariance Type:            nonrobust                                         
==============================================================================================
                                 coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      1.5222      3.301      0.461      0.645      -4.980       8.025
Urban_Population[T.10.118]     1.8789      4.669      0.402      0.688      -7.317      11.075
Urban_Population[T.10.163]     1.9524      4.669      0.418      0.676      -7.244      11.149
Urban_Population[T.10.376]     2.3627      4.669      0.506      0.613      -6.834      11.559
Urban_Population[T.10.4]       2.2957      4.669      0.492      0.623      -6.901      11.492
Urban_Population[T.10.433]    -0.6196      4.669     -0.133      0.895      -9.816       8.577
Urban_Population[T.10.641]     2.0473      4.669      0.438      0.661      -7.149      11.244
Urban_Population[T.10.642]     2.8671      4.669      0.614      0.540      -6.329      12.063
Urban_Population[T.10.796]     0.0518      4.669      0.011      0.991      -9.144       9.248
Urban_Population[T.10.883]    -0.3530      4.669     -0.076      0.940      -9.549       8.843
Urban_Population[T.10.915]     2.7991      4.669      0.600      0.549      -6.397      11.995
Urban_Population[T.10.934]     1.5130      4.669      0.324      0.746      -7.683      10.709
Urban_Population[T.100]        3.5660      3.320      1.074      0.284      -2.973      10.105
Urban_Population[T.11.194]     3.1096      4.669      0.666      0.506      -6.087      12.306
Urban_Population[T.11.243]     2.2981      4.669      0.492      0.623      -6.898      11.494
Urban_Population[T.11.288]     4.4446      4.669      0.952      0.342      -4.752      13.641
Urban_Population[T.11.35]     -0.2776      4.669     -0.059      0.953      -9.474       8.919
Urban_Population[T.11.374]     0.4550      4.669      0.097      0.922      -8.741       9.651
Urban_Population[T.11.482]     2.1302      4.669      0.456      0.649      -7.066      11.326
Urban_Population[T.11.544]     0.2397      4.669      0.051      0.959      -8.957       9.436
Urban_Population[T.11.56]      2.7500      4.669      0.589      0.556      -6.446      11.946
Urban_Population[T.11.57]      2.4740      4.669      0.530      0.597      -6.722      11.670
Urban_Population[T.11.63]      1.9516      4.669      0.418      0.676      -7.245      11.148
Urban_Population[T.11.659]     3.7253      4.669      0.798      0.426      -5.471      12.922
Urban_Population[T.11.691]     1.9781      4.669      0.424      0.672      -7.218      11.174
Urban_Population[T.11.745]    -0.3144      4.669     -0.067      0.946      -9.511       8.882
Urban_Population[T.11.752]     2.3554      4.669      0.504      0.614      -6.841      11.552
Urban_Population[T.11.776]     1.4172      4.669      0.304      0.762      -7.779      10.613
Urban_Population[T.11.79]     -0.7813      4.669     -0.167      0.867      -9.978       8.415
Urban_Population[T.11.834]    -0.2092      